# Final Transformation and Visualization of the transformed data 

In [0]:
df = spark.table('silver.transformed_data')
display(df)

In [0]:
display(df.groupBy("Company").count().show(100))

In [0]:
companies_to_exclude = [
    "UT", "CNES", "KCST", "Landspace", "SRC", "ISA", "Douglas", "CAS Space",
    "ASI", "CECLES", "Virgin Galactic", "IRGC", "JAXA", "KARI", "EER", "Arm�e de l'Air",
    "i-Space", "OneSpace", "Galactic Energy", "Firefly", "IAI", "CASIC", "Khrunichev",
    "MITT", "AMBA", "AEB", "Virgin Orbit", "Land Launch", "RAE", "OKB-586", "Yuzhmash",
    "GK LS", "Astra", "Exos", "Eurockot","Starsem","US Navy","ESA","ExPace","Blue Origin",
    "Kosmotras","Rocket Lab","ISAS","Sea Launch","ILS","Roscosmos","Lockheed","MHI","Northrop","RVSN USSR","CASC"
]

df = df.filter(~df["Company"].isin(companies_to_exclude))
display(df.groupBy("Company").count())



### Implementing of _**Gold**_ Layer in Medallion Architecture 

In [0]:
df.write.format("delta").mode("overwrite").save("/mnt/gold/target/")

In [0]:
df.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("gold.target_data")

In [0]:
df = spark.table("gold.target_data")
display(df)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### Creating the SpaceX Dataset

In [0]:
from pyspark.sql import functions as F
# Filter SpaceX launches
space_x_data = df.filter(df["Company"] == "SpaceX")

# Step 2: Group by Rocket and calculate total + successful launches
grouped_by_rocket = space_x_data.groupBy("Rocket","Country","City").agg(
    F.count("*").alias("Total_Launches"),
    F.count(F.when(F.col("MissionStatus") == "Success", True)).alias("Successful_Launches")
)

# Step 3: Calculate success rate
grouped_by_rocket = grouped_by_rocket.withColumn(
    "Success_Rate",
    (F.col("Successful_Launches") / F.col("Total_Launches"))*100
)

display(grouped_by_rocket.printSchema())

In [0]:
grouped_by_rocket.write.format("delta").mode("overwrite").save("/mnt/gold/space_x_rockets")

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
grouped_by_rocket.write.format("delta").mode("overwrite").saveAsTable("gold.space_x_rockets")

In [0]:
space_x_rockets = spark.table("gold.space_x_rockets")
display(space_x_rockets)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.